In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
import time
import os

In [2]:
housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [7]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])

model.compile(loss="mean_squared_error", optimizer="sgd")

history = model.fit(X_train, y_train, epochs=20, 
                    validation_data=(X_valid, y_valid))

Epoch 1/20
363/363 [==============================] - 4s 5ms/step - loss: nan - val_loss: nan
Epoch 2/20
363/363 [==============================] - 1s 3ms/step - loss: nan - val_loss: nan
Epoch 3/20
363/363 [==============================] - 1s 3ms/step - loss: nan - val_loss: nan
Epoch 4/20
363/363 [==============================] - 1s 3ms/step - loss: nan - val_loss: nan
Epoch 5/20
363/363 [==============================] - 1s 3ms/step - loss: nan - val_loss: nan
Epoch 6/20
363/363 [==============================] - 1s 3ms/step - loss: nan - val_loss: nan
Epoch 7/20
363/363 [==============================] - 1s 3ms/step - loss: nan - val_loss: nan
Epoch 8/20
363/363 [==============================] - 1s 3ms/step - loss: nan - val_loss: nan
Epoch 9/20
363/363 [==============================] - 1s 3ms/step - loss: nan - val_loss: nan
Epoch 10/20
363/363 [==============================] - 1s 3ms/step - loss: nan - val_loss: nan
Epoch 11/20
363/363 [==============================] - 1s 3

In [8]:
mse_test = model.evaluate(X_test, y_test)

162/162 [==============================] - 1s 3ms/step - loss: nan


In [9]:
X_new = X_test[:3] # pretend these are new instances
y_pred = model.predict(X_new)

#  Functional API

In [5]:
input = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation="relu")(input)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.Concatenate()([input, hidden2])
output = keras.layers.Dense(1)(concat)
model = keras.models.Model(inputs=[input], outputs=[output])

In [6]:
model.compile(loss="mean_squared_error", optimizer="sgd")
history = model.fit(X_train, y_train, epochs=20, 
                    validation_data=(X_valid, y_valid))

Epoch 1/20
363/363 [==============================] - 2s 4ms/step - loss: nan - val_loss: nan
Epoch 2/20
363/363 [==============================] - 1s 3ms/step - loss: nan - val_loss: nan
Epoch 3/20
363/363 [==============================] - 1s 3ms/step - loss: nan - val_loss: nan
Epoch 4/20
363/363 [==============================] - 1s 3ms/step - loss: nan - val_loss: nan
Epoch 5/20
363/363 [==============================] - 1s 3ms/step - loss: nan - val_loss: nan
Epoch 6/20
363/363 [==============================] - 1s 3ms/step - loss: nan - val_loss: nan
Epoch 7/20
363/363 [==============================] - 1s 3ms/step - loss: nan - val_loss: nan
Epoch 8/20
363/363 [==============================] - 1s 3ms/step - loss: nan - val_loss: nan
Epoch 9/20
363/363 [==============================] - 1s 3ms/step - loss: nan - val_loss: nan
Epoch 10/20
363/363 [==============================] - 1s 3ms/step - loss: nan - val_loss: nan
Epoch 11/20
363/363 [==============================] - 1s 3

In [7]:
input_A = keras.layers.Input(shape=[5])
input_B = keras.layers.Input(shape=[6])
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])

output = keras.layers.Dense(1)(concat)
aux_output = keras.layers.Dense(1)(hidden2)

model = keras.models.Model(inputs=[input_A, input_B],
                            outputs=[output, aux_output])

In [ ]:
model.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer="sgd")

X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

history = model.fit([X_train_A, X_train_B],
                    [y_train, y_train],
                    epochs=20,
                    validation_data=([X_valid_A, X_valid_B], [y_valid, y_valid]))

total_loss, main_loss, aux_loss = model.evaluate([X_test_A, X_test_B], [y_test, y_test])

In [11]:
y_pred = model.predict((X_new_A, X_new_B))

# Subclass API

In [2]:
class WideAndDeepModel(keras.models.Model):
    # you define the layers here
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs) # handles standard args (e.g., name)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)
    
    # you can connect the layer here, the advantage is that you can use loops, if statementes, logic operations etc.
    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output

model = WideAndDeepModel()

# Checkpoints

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])

model.compile(loss="mean_squared_error", optimizer="sgd")



checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5",
                                                save_best_only=True) # if using validation set, will save the model with the best score on validation


early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                                  restore_best_weights=True)

history = model.fit(X_train, y_train, epochs=100,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb, early_stopping_cb]) # this will both save the model after each epoch and stop training if the model does not improve anymore


### Create custom callback functions

In [ ]:
class PrintValTrainRatioCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        print("\nval/train: {:.2f}".format(logs["val_loss"] / logs["loss"]))
        
print_cb = PrintValTrainRatioCallback()
history = model.fit(X_train, y_train, epochs=100,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb, early_stopping_cb, print_cb]) # in addition to the checkpoint and early stopping callback, this will print the val/train ratio after each epoch

# TensorBoard

In [3]:
root_logdir = os.path.join(os.curdir, "my_logs")

def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()   

In [4]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

model.compile(loss="mean_squared_error", optimizer="adam")

tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid),
                    callbacks=[tensorboard_cb])

Epoch 1/10
363/363 [==============================] - 3s 5ms/step - loss: 2.5078 - val_loss: 2.3855
Epoch 2/10
363/363 [==============================] - 2s 4ms/step - loss: 2.4747 - val_loss: 2.3855
Epoch 3/10
363/363 [==============================] - 2s 4ms/step - loss: 2.4747 - val_loss: 2.3855
Epoch 4/10
363/363 [==============================] - 1s 4ms/step - loss: 2.4747 - val_loss: 2.3855
Epoch 5/10
363/363 [==============================] - 1s 4ms/step - loss: 2.4747 - val_loss: 2.3855
Epoch 6/10
363/363 [==============================] - 1s 4ms/step - loss: 2.4747 - val_loss: 2.3855
Epoch 7/10
363/363 [==============================] - 2s 4ms/step - loss: 2.4747 - val_loss: 2.3855
Epoch 8/10
363/363 [==============================] - 1s 4ms/step - loss: 2.4747 - val_loss: 2.3855
Epoch 9/10
363/363 [==============================] - 1s 4ms/step - loss: 2.4747 - val_loss: 2.3855
Epoch 10/10
363/363 [==============================] - 1s 4ms/step - loss: 2.4747 - val_loss: 2.3855

# GridSearch

In [7]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    options = {"input_shape": input_shape}
    
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu", **options))  
        options = {}
        
    model.add(keras.layers.Dense(1, **options))
    optimizer = keras.optimizers.SGD(learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [8]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

C:\Users\hasan\AppData\Local\Temp/ipykernel_4068/1709004121.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)


### You can use the model as a scikit-learn model

In [ ]:
keras_reg.fit(X_train, y_train, epochs=100,
              validation_data=(X_valid, y_valid), 
              callbacks=[keras.callbacks.EarlyStopping(patience=10)])

mse_test = keras_reg.score(X_test, y_test)
y_pred = keras_reg.predict(X_new)

In [ ]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(3e-4, 3e-2),
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3)

rnd_search_cv.fit(X_train, y_train, epochs=100,
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

In [ ]:
# best parameters
rnd_search_cv.best_params_
# best score with those parameters
rnd_search_cv.best_score_
# best model
model = rnd_search_cv.best_estimator_.model